In [13]:
#pip install spark-nlp
#pip install pyspark
#pip install --upgrade pymupdf

import pymupdf

def pdf_reader(): #test method for reading pdfs
    #open file
    file = pymupdf.open("Merged March 2023 (wine).pdf")
    
    #read the file
    out = open("output.txt", "wb")
    for page in file: #iterate the pages
        text = page.get_text().encode("utf8") #get plain text
        out.write(text) #write text of page
        out.write(bytes((12,))) #write page delimiter (form feed 0x0C)
    print(out)
        
    out.close
    return

def txt_reader(): #test method for reading text files
    #open file
    file = open("testerfile.txt")
    
    #read the file
    content = file.read()
    print(content)
    
    #close the file
    file.close()
    return




In [14]:
def main():
    #doc = pymupdf.open("")
    #txt_reader()
    pdf_reader()
    return

if __name__ == "__main__":
    main()

<_io.BufferedWriter name='output.txt'>
